# Projet : Traitement Automatique des Langues (Partie 1)

Nous avons à notre disposition deux fichiers CSV ([allocine_genres_test.csv](data/allocine_genres_test.csv) et [allocine_genres_train.csv](data/allocine_genres_train.csv)) contenant des informations sur des films et leurs genres. Le but de ce projet est de prédire les genres d'un film à partir de son synopsis notamment (et d'autres informations).

L’objectif est d’entraîner un outil de classification automatique des films en fonction de leur genre. La classification doit se baser sur le texte de la synopsis et sur le titre des films. Le texte et le titre des articles ont déjà été tokenisés et tous les tokens sont séparés par un espace.

## 1. Importation des données et analyse exploratoire

Les données sont disponibles dans le dossier [data](data/). Nous allons commencer par importer les données et les analyser.

In [ ]:
%%capture

# exécuter cette cellule pour installer les dépendances et télécharger les modèles spacy
# remplacer `python` par `python3` si nécessaire
# !conda create -n nlp python=3.11
# !conda activate nlp
!python -m pip install --upgrade -r requirements.txt
!python -m spacy download fr_core_news_sm

In [ ]:
import os
import sys

import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
import spacy

from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from spacy import displacy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.linear_model import LogisticRegression

train: pd.DataFrame = pd.read_csv(os.path.join('data', 'allocine_genres_train.csv'))

In [ ]:
print(train.shape)
print(train.head())

In [ ]:
print(train['genre'].value_counts(), end='\n\n')
print(random.choice(train['synopsis'].unique()))

On remarque déjà que les donnés chiffrées sont soit des entiers soit des flottants. Les données textuelles sont des chaînes de caractères. Certaines données sont manquantes : `NaN` dans le cas des données chiffrées et une chaîne vide dans le cas des données textuelles.

Pour traiter les données manquantes, nous avons deux solutions :

1. Dans le cas où il y a très peu de données manquantes, on peut simplement supprimer les entrées qui contiennent ces données manquantes. Cela peut être acceptable si le nombre de données manquantes est très faible par rapport à la taille de l'ensemble de données et que la suppression de ces entrées n'affecte pas significativement les résultats de l'analyse.
2. En revanche, si le nombre de données manquantes est important, la suppression de ces entrées pourrait entraîner une perte d'informations importantes pour l'analyse. Dans ce cas, il est généralement préférable de remplacer les valeurs manquantes par une valeur qui représente au mieux l'information manquante. Par exemple, si les données manquantes sont des scores au box-office, vous pouvez remplacer ces données manquantes par la moyenne ou la médiane des scores de box-office disponibles dans les données.


In [ ]:
EMPTY_TOKEN = '<EMPTY>'

# remove rows with missing values
train_dataset_1 = train.dropna(how='any', inplace=False)

# replace missing values with either the mean or the median (or empty token)
list_headers = train.columns.values.tolist()
list_of_numerical_headers = train._get_numeric_data().columns.values.tolist()
list_of_categorical_headers = list(set(list_headers) - set(list_of_numerical_headers))

train_dataset_2 = train.copy()
for header in list_of_numerical_headers:
  train_dataset_2[header].fillna(train_dataset_2[header].median(), inplace=True)
for header in list_of_categorical_headers:
  train_dataset_2[header].fillna(EMPTY_TOKEN, inplace=True)

print(train_dataset_1.shape)
print(train_dataset_2.shape)

Lorsqu'on supprime simplement toutes les entrées où il manque au moins une valeur, on se retrouve uniquement avec 891 valeurs en tout. Cela signifie que nous avons perdu beaucoup d'informations. Nous allons donc utiliser la deuxième solution (au moins dans un premier temps) et remplacer les valeurs manquantes par des valeurs qui représentent au mieux l'information manquante.

## 2. Prétraitement des données

Il faut aussi corriger les entrées textuelles, ainsi qu'appliquer un certain nombre d'algorithmes de prétraitement comme : la suppression des caractères spéciaux, la suppression des stop words, la suppression des mots trop fréquents ou trop rares, la lemmatisation, la suppression des mots trop longs, etc.

On définit donc un ensemble de fonctions et de filtres qui vont nous permettre de prétraiter les données textuelles.

In [ ]:
nltk.download('stopwords', quiet=True)        # download the stopwords corpus
nlp = spacy.load('fr_core_news_sm')           # load the French model
fr_stopwords = set(stopwords.words('french')) # so that `in` tests are faster
stemmer = FrenchStemmer()                     # for stemming words


# get the tokens of a sentence (word based tokenization)
def get_tokens_words(text: str) -> list[str]:
  doc = nlp(text)
  return [w.text for w in doc]


# remove stopwords from a sentence
def clean_sentence(text: str) -> list[str]:
  clean_words: list[str] = []
  for token in get_tokens_words(text):
    if token not in fr_stopwords:
      clean_words.append(token)
  return clean_words


# get the tokens of multiple sentences (sentence based tokenization)
def get_tokens_sentences(text: str) -> list[str]:
  doc = nlp(text)
  return [s.text for s in doc.sents]


# get the lemmas of a sentence
def get_stem(text: str) -> list[str]:
  doc = nlp(text)
  return [stemmer.stem(w.text) for w in doc]


# get the named entities of a sentence
def get_ner(text: str) -> list[str]:
  doc = nlp(text)
  return [(ent.text, ent.label_) for ent in doc.ents]


# render the named entities of a sentence in a Jupyter notebook
def render_ner(text: str) -> None:
  doc = nlp(text)
  displacy.render(doc, style='ent', jupyter=True)


# get the part of speech of a sentence
def get_pos(text: str) -> list[str]:
  doc = nlp(text)
  return [(token, token.pos_) for token in doc]


# render the part of speech of a sentence in a Jupyter notebook
def render_pos(text: str) -> None:
  doc = nlp(text)
  displacy.render(doc, style='dep', options={'distance': 90})


# get the word embeddings of a sentence
def get_word_embeddings(text: str) -> list[np.ndarray]:
  doc = nlp(text)
  return [token.vector for token in doc]


# get the similarity between two sentences
def get_mean_embedding(text1: str, text2: str) -> float:
  doc1 = nlp(text1)
  doc2 = nlp(text2)
  mean1 = np.mean([token.vector for token in doc1], axis=0)
  mean2 = np.mean([token.vector for token in doc2], axis=0)

  return np.dot(mean1, mean2) / (np.linalg.norm(mean1) * np.linalg.norm(mean2))

Juste pour clarifier les choses, nous allons simplement effectuer des tests sur l'ensemble de phrases suivantes :

1. "Le réseau sera bientôt rétabli à Marseille"
2. "La panne réseau affecte plusieurs utilisateurs de l'opérateur"
3. "Il fait 18 degrés ici"
4. "Bouygues a eu une coupure de réseau à Marseille. La panne a affecté 300.000 utilisateurs."

In [ ]:
text1 = 'Le réseau sera bientôt rétabli à Marseille.'
text2 = 'La panne réseau affecte plusieurs utilisateurs de l\'opérateur'
text3 = 'Il fait 18 degrés ici'
text4 = 'Bouygues a eu une coupure de réseau à Marseille. La panne a affecté 300.000 utilisateurs.'

In [ ]:
# basic tokenization
# we can observe `get_tokens_sentences` do not "cut" at each . or ! or ?

print(get_tokens_words(text1))
print(clean_sentence(text1))
print(get_tokens_sentences(text4))

In [ ]:
# stemming
# this doesn't work very well for French...

print(get_stem(text1))
print(get_stem(text2))

In [ ]:
# named entities recognition

print(get_ner(text4))
render_ner(text4)

In [ ]:
# part of speech

print(get_pos(text1))
render_pos(text1)

In [ ]:
# word embeddings and mean embedding (similarity)

print(get_word_embeddings(text1)[0].shape)
print(get_mean_embedding(text1, text2))
print(get_mean_embedding(text1, text4))
print(get_mean_embedding(text2, text4))

## 3. Préparation des données

Nous allons maintenant préparer les données pour l'entraînement de notre modèle.

Nous allons donc appliquer les fonctions de prétraitement sur les données textuelles et transformer les données chiffrées en données numériques. Dans un premier temps, nous confectionnerons des ensembles contenant toutes les valeurs et les informations présentes dans les données. Nous verrons par la suites lesquelles sont les plus pertinantes en fonction des résultats obtenus et de nos modèles.

In [ ]:
# we will use train_dataset_2 since it has no missing values

values = ['synopsis', 'titre']
X = train_dataset_2[values]
y = train_dataset_2['genre']

print(X.shape)
print(X.head())
print(y.shape)
print(y.head())

In [ ]:
# separate the dataset into a training set and a validation set
X_train, y_train = X, y

Pour la suite, nous définirons des pipelines de traitement spécifiques à chaque type de colonne. En particulier, les colonnes correspondant à des textes (`list_of_categorical_headers`) dans `X` seront vectorisées.

Pour toutes les données dans `list_of_categorical_headers`, les tokens ont déjà été séparés par des espaces. Nous allons utiliser `TfidfVectorizer` pour vectoriser ces données.

In [ ]:
tfidf_vectorizer = TfidfVectorizer(
  analyzer='word',
  tokenizer=lambda x: str.split(x, sep=' '),
  token_pattern=None,
  lowercase=True,
  stop_words=list(fr_stopwords),
  min_df=0.01,
  max_df=0.95,
)


In [ ]:
res = tfidf_vectorizer.fit_transform(X_train['synopsis'])
bow = pd.DataFrame(res.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
print(bow.shape)
print(bow.head())

Nous pouvons aussi utiliser ce `vectorizer` pour extraire des statistiques sur les données textuelles. Par exemple, la longueur en nombre de caractères, le nombre de phrases, ...

In [ ]:
def make_stats(texts: list[str]) -> list[dict[str, int]]:
  return [{
    'len': len(t),
    'nb_sentences': t.count('.') + t.count('!') + t.count('?'),
  } for t in texts]

stats_transformer = FunctionTransformer(make_stats, validate=False)
stats_vectorizer = DictVectorizer(sparse=False)

In [ ]:
res = stats_vectorizer.fit_transform(stats_transformer.transform(X_train['synopsis']))
stats = pd.DataFrame(res, columns=stats_vectorizer.get_feature_names_out())

print(stats.shape)
print(stats.head())

On normalise les données en utilisant `MinMaxScaler` pour les données dans notre dictionnaire

In [ ]:
min_max_scaler = MinMaxScaler()
scaled_stats = pd.DataFrame(min_max_scaler.fit_transform(stats), columns=stats.columns)

print(scaled_stats.shape)
print(scaled_stats.head())

On rajoute aussi `CountVectorizer` ainsi que `HashingVectorizer` pour voir si les résultats sont meilleurs.

In [ ]:
count_vectorizer = CountVectorizer(
  analyzer='word',
  tokenizer=lambda x: str.split(x, sep=' '),
  token_pattern=None,
  lowercase=True,
  stop_words=list(fr_stopwords),
  min_df=0.01,
  max_df=0.95,
)
hashing_vectorizer = HashingVectorizer(
  analyzer='word',
  tokenizer=lambda x: str.split(x, sep=' '),
  token_pattern=None,
  lowercase=True,
  stop_words=list(fr_stopwords),
  n_features=1000,
)

In [ ]:
res = count_vectorizer.fit_transform(X_train['synopsis'])
bow = pd.DataFrame(res.toarray(), columns=count_vectorizer.get_feature_names_out())

print(bow.shape)
print(bow.head())

In [ ]:
res = hashing_vectorizer.fit_transform(X_train['synopsis'])
bow = pd.DataFrame(res.toarray(), columns=[f'feature_{i}' for i in range(1000)])

print(bow.shape)
print(bow.head())

## 4. Création de la pipeline

Nous allons maintenant procéder à la création de la pipeline en combinant les chaînes de pré-traitement.

In [ ]:
column_transformer = ColumnTransformer(
  [
    # 'synopsis' column : tf-idf vectorization
    ('synopsis', tfidf_vectorizer, 'synopsis'),
    # 'titre' column : tf-idf vectorization
    ('titre', tfidf_vectorizer, 'titre'),

    # 'synopsis' column : stats
    ('synopsis_stats', Pipeline([
      ('stats_transformer', stats_transformer),
      ('stats_vectorizer', stats_vectorizer),
      ('min_max_scaler', min_max_scaler),
    ]), 'synopsis'),

  ],
  remainder='drop', # drop the columns not specified
)

In [ ]:
# learning
classifier_pipeline = make_pipeline(column_transformer, LogisticRegression(max_iter=1000))

In [ ]:
# fit
classifier_pipeline.fit(X_train, y_train)

In [ ]:
# predict
test: pd.DataFrame = pd.read_csv(os.path.join('data', 'allocine_genres_test.csv'))

X_test = test[values]
y_test = test['genre']

y_pred = classifier_pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

## 5. Validation croisée

Nous allons maintenant procéder à la validation croisée de notre modèle. Nous allons utiliser `GridSearchCV` pour tester plusieurs hyperparamètres et choisir les meilleurs.

Les imports nécessaires sont effectués dans la cellule suivante.

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

models = [
  ('Baseline', DummyClassifier(strategy='most_frequent')),
  ('Mutinomial NB', MultinomialNB()),
  ('CART', DecisionTreeClassifier()),
  ('LR', LogisticRegression()),
  ('KNN', KNeighborsClassifier()),
  ('Random forest', RandomForestClassifier()),
]

# do cross validation
for name, model in models:
  print(name)
  pipeline = make_pipeline(column_transformer, model)
  scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='f1_macro')
  print(f'  {scores.mean():.3f} +/- {scores.std():.3f}')

# do grid search on the best model
from sklearn.model_selection import GridSearchCV

param_grid = {
  'logisticregression__C': [0.1, 1, 10],
  'logisticregression__solver': ['lbfgs', 'liblinear'],
}

grid_search = GridSearchCV(classifier_pipeline, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)
print(grid_search.cv_results_)


## 6. Passage sur les transformers

Nous allons maintenant explorer la piste des transformers. Pour des raisons de reproductibilité, nous allons utiliser des modèles pré-entraînés. Tous les imports nécessaires seront effectués sur les cellules suivantes.

Nous allons utiliser le modèle `xlm-roberta-large-xnli` pour la classification de nos données. Nous avons besoin pour cela de nouveaux packages :

- `transformers`
- `torch`
- `protobuf`
- `sentencepiece`

In [1]:
import os
import sys

import torch
import pandas as pd
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

uri = 'BaptisteDoyen/camembert-base-xnli'
tokenizer = AutoTokenizer.from_pretrained(uri)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained(uri).to(device)

train: pd.DataFrame = pd.read_csv(os.path.join('data', 'allocine_genres_train.csv'))

classifier = pipeline('zero-shot-classification', model=model, tokenizer=tokenizer)

candidate_labels: list[str] = train['genre'].unique().tolist()
hypothesis_template: str = "Ce film est du genre {}."


/home/tbyr/miniconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def predict_genre(text: str) -> str:
  result = classifier(text, candidate_labels, hypothesis_template=hypothesis_template)
  return result['labels'][0]

In [3]:
EMPTY_TOKEN = '<EMPTY>'

# replace missing values with either the mean or the median (or empty token)
list_headers = train.columns.values.tolist()
list_of_numerical_headers = train._get_numeric_data().columns.values.tolist()
list_of_categorical_headers = list(set(list_headers) - set(list_of_numerical_headers))

train_dataset_2 = train.copy()
for header in list_of_numerical_headers:
  train_dataset_2[header].fillna(train_dataset_2[header].median(), inplace=True)
for header in list_of_categorical_headers:
  train_dataset_2[header].fillna(EMPTY_TOKEN, inplace=True)

print(train_dataset_2.shape)
X_train, y_train = train_dataset_2['synopsis'], train_dataset_2['genre']

(2875, 22)


In [5]:
# make model
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

import tqdm

class GenrePredictor(BaseEstimator, TransformerMixin):

  def __init__(self, classifier):
    self.classifier = classifier # pipeline

  def fit(self, X, y):
    return self # nothing else to do

  def predict(self, X):
    return [predict_genre(x) for x in tqdm.tqdm(X)]


training_args = TrainingArguments(
  output_dir='./results',          # output directory
  num_train_epochs=1,              # total number of training epochs
  per_device_train_batch_size=16,  # batch size per device during training
  per_device_eval_batch_size=64,   # batch size for evaluation
  warmup_steps=500,                # number of warmup steps for learning rate scheduler
  weight_decay=0.01,               # strength of weight decay
  logging_dir='./logs',            # directory for storing logs
  logging_steps=10,
)

# todo: make train and eval datasets
trainer = Trainer(
  model=model,                         # the instantiated 🤗 Transformers model to be trained
  args=training_args,                  # training arguments, defined above
  train_dataset=train_dataset_2,       # training dataset
  eval_dataset=train_dataset_2,        # evaluation dataset
)

# learning
classifier_pipeline = make_pipeline(GenrePredictor(classifier))

# fit
print(X_train.shape)
classifier_pipeline.fit(X_train, y_train)

# predict
test: pd.DataFrame = pd.read_csv(os.path.join('data', 'allocine_genres_test.csv'))

X_test = test['synopsis']
y_test = test['genre']

print(X_test.shape)
y_pred = classifier_pipeline.predict(X_test) # takes about 8 minutes (i9-13900h, 32GB RAM)


(2875,)
(719,)


100%|██████████| 719/719 [09:08<00:00,  1.31it/s]


In [9]:
print(X_test)
print(y_pred)
print(y_test.to_list())

print(classification_report(y_test, y_pred))

0      Gilbert Grape vit à Endora dans l' Iowa , avec...
1      Aventure à la fois complexe et mystérieuse sur...
2      A la mort de sa mère , Anne fait une découvert...
3      Christiane , une jeune berlinoise de treize an...
4      Après son voyage mouvementé entre passé , prés...
                             ...                        
714    Un jeune ténor du barreau gagne toutes les cau...
715    Un biopic du Président des Etats-Unis Theodore...
716    Deux astronautes , le Lieutenant Payton et le ...
717    L' histoire vraie d' une avocate défendant une...
718    A Casablanca , pendant la Seconde Guerre mondi...
Name: synopsis, Length: 719, dtype: object
['historique', 'historique', 'historique', 'drame', 'historique', 'drame', 'drame', 'policier', 'romance', 'comédie', 'historique', 'historique', 'drame', 'comédie', 'historique', 'policier', 'historique', 'policier', 'policier', 'drame', 'historique', 'documentaire', 'horreur', 'historique', 'comédie', 'policier', 'drame', 'pol